Define search function
Subtask:
Create a function that takes a name as input and constructs the search URL.

In [1]:
def construct_kenyalaw_search_url(name):
  """Constructs the search URL for Kenya Law with the provided name.

  Args:
    name: The name to search for.

  Returns:
    The constructed search URL.
  """
  base_url = "https://new.kenyalaw.org/search/?q=name&nature=Judgment"
  search_url = base_url.replace("name", name)
  return search_url


Fetch search results
Subtask:
Use the requests library to fetch the HTML content of the search results page.

In [2]:
import requests

search_name = "Agnes+Kagure"
search_url = construct_kenyalaw_search_url(search_name)
response = requests.get(search_url)

if response.status_code == 200:
    print("Request successful.")
else:
    print(f"Request failed with status code: {response.status_code}")

Request successful.


Install Selenium and Wbdriver-Manager

In [4]:
%pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.7 MB 363.1 kB/s eta 0:00:27
   ---------------------------------------- 0.1/9.7 MB 393.8 kB/s eta 0:00:25
   ---------------------------------------- 0.1/9.7 MB 393.8 kB/s eta 0:00:25
   ---------------------------------------- 0.1/9.7 MB 328.0 kB/s eta 0:00:30
    --------------------------------------- 0.2/9.7 MB 499.5 kB/s eta 0:00:20
    --------------------------------------- 0.2/9.7 MB 565.6 kB/s eta 0:00:17
    --------------------------------------- 0.2/9.7 MB 529.7 kB/s eta 0:00:18
    --------------------------------------- 0.2/9.7 MB 529.7 kB/s eta 0:00:18
   - -------------------------------------- 0.3/9.7 MB 535.8 kB/s eta 0:00:18
   - -------------------------------------- 0.4/9.7 MB 654.8 kB/s eta 0:00:15
   - --------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Fetch dynamic content
Subtask:
Use Selenium to navigate to the search URL and wait for the search results to load.

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Instantiate the Chrome webdriver with the defined options
driver = webdriver.Chrome(options=options)

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

search_name = "agnes kagure"
search_url = construct_kenyalaw_search_url(search_name)

driver.get(search_url)

# Wait for the search results container to be present
try:
    search_results_container = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-v-1dac1b9c]'))
    )
    print("Search results container found.")
except:
    print("Search results container not found within the waiting time.")

Search results container found.


Parse dynamic content
Subtask:
Get the page source after the dynamic content has loaded and use BeautifulSoup to parse it.

In [8]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

Extract and display results
Subtask:
Extract the links and titles of the judgments from the parsed HTML and display them.

In [9]:
search_results_container = soup.select_one('div[data-v-1dac1b9c]')

if search_results_container:
    results_list = search_results_container.select_one('ul.list-unstyled')

    if results_list:
        judgment_links = results_list.select('a')
        print("Search Results:")
        for link in judgment_links:
            href = link.get('href')
            title = link.text.strip()
            print(f"Title: {title}\nLink: {href}\n")
    else:
        print("No search results list found on the page.")
else:
    print("No search results container found on the page.")


Search Results:
Title: Uniglobe Northline Travel Limited v Agnes Kagure Kariuki [2021] KEHC 3264 (KLR)
Link: /akn/ke/judgment/kehc/2021/3264/eng@2021-10-01

Title: Uniglobe Northline Travel Limited v Agnes Kagure Kariuki [2018] KEHC 8937 (KLR)
Link: /akn/ke/judgment/kehc/2018/8937/eng@2018-01-30

Title: Uniglobe Northline Travel Ltd v Agnes Kagure Kariuki & another [2018] KEHC 1682 (KLR)
Link: /akn/ke/judgment/kehc/2018/1682/eng@2018-11-02

Title: Uni Globe Northline Travel Ltd v Agnes Kagure Kariuki & another [2018] KEHC 1769 (KLR)
Link: /akn/ke/judgment/kehc/2018/1769/eng@2018-12-14

Title: Joel Munyoki Munene v Agnes Kagure Kariuki & 2 others [2017] KEELC 284 (KLR)
Link: /akn/ke/judgment/keelc/2017/284/eng@2017-12-05

Title: Ruth Wambui Kimani(suing as an Administratix of the Estate of the lateFrancis Kimani Mungui & another v Agnes Kagure Kariuki & 4 others [2022] KEELC 562 (KLR)
Link: /akn/ke/judgment/keelc/2022/562/eng@2022-03-24

Title: Joel Munyoki Munene v Agnes Kagure Kariuki

Analyze search results
Subtask:
Categorize or count the types of judgments based on keywords in the titles.

In [10]:
# Define keywords for categorization
judgment_categories = {
    "High Court": ["KEHC"],
    "Environment and Land Court": ["KEELC"],
    "Court of Appeal": ["KECA"],
    # Add more categories and keywords as needed
}

# Categorize judgments based on keywords in the title
categorized_judgments = {}
for link in judgment_links:
    title = link.text.strip()
    category = "Other"  # Default category
    for cat, keywords in judgment_categories.items():
        if any(keyword in title for keyword in keywords):
            category = cat
            break
    if category not in categorized_judgments:
        categorized_judgments[category] = []
    categorized_judgments[category].append({"title": title, "link": link.get('href')})

# Display the categorized judgments and counts
print("\nCategorized Judgments:")
for category, judgments in categorized_judgments.items():
    print(f"\n--- {category} ({len(judgments)}) ---")
    for judgment in judgments:
        print(f"Title: {judgment['title']}")
        print(f"Link: {judgment['link']}")


Categorized Judgments:

--- High Court (5) ---
Title: Uniglobe Northline Travel Limited v Agnes Kagure Kariuki [2021] KEHC 3264 (KLR)
Link: /akn/ke/judgment/kehc/2021/3264/eng@2021-10-01
Title: Uniglobe Northline Travel Limited v Agnes Kagure Kariuki [2018] KEHC 8937 (KLR)
Link: /akn/ke/judgment/kehc/2018/8937/eng@2018-01-30
Title: Uniglobe Northline Travel Ltd v Agnes Kagure Kariuki & another [2018] KEHC 1682 (KLR)
Link: /akn/ke/judgment/kehc/2018/1682/eng@2018-11-02
Title: Uni Globe Northline Travel Ltd v Agnes Kagure Kariuki & another [2018] KEHC 1769 (KLR)
Link: /akn/ke/judgment/kehc/2018/1769/eng@2018-12-14
Title: Republic v Director of Public Prosecutions & another; Ex parte Guy Spencer Elms & another; Agnes Kagure Kariuki (Interested Party) [2021] KEHC 7915 (KLR)
Link: /akn/ke/judgment/kehc/2021/7915/eng@2021-03-17

--- Environment and Land Court (5) ---
Title: Joel Munyoki Munene v Agnes Kagure Kariuki & 2 others [2017] KEELC 284 (KLR)
Link: /akn/ke/judgment/keelc/2017/284/eng